In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import trange
from tqdm.notebook import tqdm
from scipy.signal import correlate
import os
import scipy as sp
import scipy.optimize

from isingpackage import*

In [8]:
def heat_capacity(T, N, measurements=2000, inverse=False):
    """heat_capacity measures the heat capacity of a lattice with a given size N under given temperature

    Args:
        T (float): temperature
        N (int): square lattice width
        measurements (int, optional): Number of frames that we're taking the variance of the energy with. Defaults to 2000.
        inverse (bool, optional): If true, return the reciprical of C. Defaults to False.

    Returns:
        float: Heat capacity measured
    """
    #create a random 
    seed = np.random.randint(1000)
    array = spin_array(N=N, randomseed = seed)

    eq_wait = 1000

    #waiting the system to reach equilbrium from the initial random state
    for i in range(eq_wait):
        array.update_array(T=T)


    E_list = []

    #recording the total energy of each frames
    for i in range(measurements):
        array.update_array(T=T)
        E_list.append(array.get_E())

    C = np.var(E_list)/(T**2)

    #inversed solution can be used to find the minimum of the inversed function
    #using the golden section method
    if inverse:
        return 1/C
    else:
        return C


In [ ]:
#This cell carry out the measurements and save it in a csv file

Ns = [5,10,15,20,25,32]

#Measuring the heat capacity of different lattice sizes.
for N in Ns:

    print('N={}'.format(N))
    
    #total frames for each measurement
    frames=1000

    #amount of measurements we do for each lattice size at each temperature
    repeat = 20

    string = ''

    Ts = np.linspace(1,3, 100)

    for T in tqdm(Ts):

        string += '{} '.format(T)

        for i in range(repeat):

            C = heat_capacity(N=N, T=T, measurements=frames, max_tries=100)

            string = string + ', ' + str(C) 

        string += '\n'

        with open('measurements\\C_{}.csv'.format(N), 'w') as f:
            f.write(string)